## Ejercicio 58

Observe una muestra aleatoria de los comentarios de las acciones realizadas por
usuarios o ips antes de ser bloqueados. Observe otra muestra de comentarios de
acciones de todos.

In [1]:
import pandas as pd
import numpy as np

In [2]:
logs = pd.read_csv("logs.csv")
logs.head()

C:\Users\alejo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,item_id,timestamp,contributor_username,contributor_id,contributor_ip,comment,logtype,action,title
0,1,2004-12-23T07:10:09Z,Sabbut,2075.0,NaN,el contenido era: 'Si aprendés Pensamiento Cie...,delete,delete,Verificacionismo
1,2,2004-12-23T07:14:09Z,Sabbut,2075.0,NaN,el contenido era: '{{destruir}}en algun lugar ...,delete,delete,Rapunzel
2,3,2004-12-23T07:26:10Z,Sabbut,2075.0,NaN,el contenido era: '#REDIRECT [[Flash]]',delete,delete,Macromedia Flash
3,4,2004-12-23T07:55:09Z,Sanbec,7856.0,NaN,Sin información de origen ni de licencia. Hay ...,delete,delete,Archivo:Gnu head.png
4,5,2004-12-23T08:02:22Z,Sanbec,7856.0,NaN,Sustituida por [[commons:Image:Heckert_GNU.png]],delete,delete,Archivo:Gnu-head-sm.jpg


Solo necesito algunas columnas, por lo que reduzco el dataframe

In [3]:
logs = logs[['timestamp','contributor_username','contributor_ip','action','title']]
logs.head()

,timestamp,contributor_username,contributor_ip,action,title
0,2004-12-23T07:10:09Z,Sabbut,NaN,delete,Verificacionismo
1,2004-12-23T07:14:09Z,Sabbut,NaN,delete,Rapunzel
2,2004-12-23T07:26:10Z,Sabbut,NaN,delete,Macromedia Flash
3,2004-12-23T07:55:09Z,Sanbec,NaN,delete,Archivo:Gnu head.png
4,2004-12-23T08:02:22Z,Sanbec,NaN,delete,Archivo:Gnu-head-sm.jpg


In [4]:
# Como trabajo con un datetime, aplico el cast
logs['timestamp'] = pd.to_datetime(logs['timestamp'], format='%Y-%m-%dT%H:%M:%SZ')
logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12188946 entries, 0 to 12188945
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   timestamp             datetime64[ns]
 1   contributor_username  object        
 2   contributor_ip        object        
 3   action                object        
 4   title                 object        
dtypes: datetime64[ns](1), object(4)
memory usage: 465.0+ MB


1 - Solo me quedo con aquellas filas cuyo action es block.

In [5]:
logs['username/ips'] = logs['contributor_username'].fillna(logs['contributor_ip'])
del logs['contributor_username']
del logs['contributor_ip']
logs

,timestamp,action,title,username/ips
0,2004-12-23 07:10:09,delete,Verificacionismo,Sabbut
1,2004-12-23 07:14:09,delete,Rapunzel,Sabbut
2,2004-12-23 07:26:10,delete,Macromedia Flash,Sabbut
3,2004-12-23 07:55:09,delete,Archivo:Gnu head.png,Sanbec
4,2004-12-23 08:02:22,delete,Archivo:Gnu-head-sm.jpg,Sanbec
...,...,...,...,...
12188941,2021-09-03 03:09:13,create,Usuario discusión:El guachicol,SeroBOT
12188942,2021-09-03 03:09:48,create,Usuario discusión:2806:2F0:3140:BE0:C09B:D1C4:...,Leonpolanco
12188943,2021-09-03 03:09:49,create,Usuario:Desiretjs,Desiretjs
12188944,2021-09-03 03:10:33,create,Contenedores,Correogsk


In [6]:
action_grouped = logs.groupby(['action'])
blocked_actions = action_grouped.get_group('block')
blocked_actions = blocked_actions[['timestamp','title']]
blocked_actions

,timestamp,title
5,2004-12-23 08:28:32,Usuario:212.55.30.157
287,2004-12-27 17:54:50,Usuario:201.241.228.15
288,2004-12-27 17:57:12,Usuario:201.255.50.32
290,2004-12-27 17:59:56,Usuario:84.122.0.158
313,2004-12-28 01:38:59,Usuario:81.192.12.93
...,...,...
12188865,2021-09-03 02:05:46,Usuario:186.141.200.121
12188886,2021-09-03 02:24:41,Usuario:Galaxi066
12188909,2021-09-03 02:52:42,Usuario:Xx.tkpro.xx
12188924,2021-09-03 03:00:14,Usuario:181.167.104.152


In [7]:
blocked_actions['title'].replace({'nan':None},inplace=True)
blocked_actions['title'].isnull().sum()

391

In [8]:
blocked_actions['title'].dropna(inplace=True)

In [9]:
blocked_actions['title'].isnull().sum()

0

In [10]:
# Con los nulls limpios, manipulo los strings de manera que solo me quede los usuarios que fueron bloqueados
def clean_Usuario(x):
    j = x.split(":")
    return j[1]

In [11]:
blocked_actions = blocked_actions.astype({'title': 'str'})


In [12]:
blocked_actions['title'].replace({'nan':None},inplace=True)
blocked_actions['title'].dropna(inplace=True)
blocked_actions['title'] = blocked_actions['title'].apply(clean_Usuario)

In [13]:
# Como title ya son los usuarios bloqueados,modifico el nombre
blocked_actions.rename(columns={'title':'user_blocked'},inplace=True)

In [14]:
blocked_actions

,timestamp,user_blocked
5,2004-12-23 08:28:32,212.55.30.157
287,2004-12-27 17:54:50,201.241.228.15
288,2004-12-27 17:57:12,201.255.50.32
290,2004-12-27 17:59:56,84.122.0.158
313,2004-12-28 01:38:59,81.192.12.93
...,...,...
12188865,2021-09-03 02:05:46,186.141.200.121
12188886,2021-09-03 02:24:41,Galaxi066
12188909,2021-09-03 02:52:42,Xx.tkpro.xx
12188924,2021-09-03 03:00:14,181.167.104.152


In [15]:
# Necesito saber cuando fue la primera vez que los usuarios que se repiten fueron bloqueados por lo que.
blocked_actions['first_block_date'] = blocked_actions.groupby('user_blocked')['timestamp'].transform('min')
blocked_actions = blocked_actions[['timestamp','user_blocked','first_block_date']]
blocked_actions

,timestamp,user_blocked,first_block_date
5,2004-12-23 08:28:32,212.55.30.157,2004-12-23 08:28:32
287,2004-12-27 17:54:50,201.241.228.15,2004-12-27 17:54:50
288,2004-12-27 17:57:12,201.255.50.32,2004-12-27 17:57:12
290,2004-12-27 17:59:56,84.122.0.158,2004-12-27 17:59:56
313,2004-12-28 01:38:59,81.192.12.93,2004-12-28 01:38:59
...,...,...,...
12188865,2021-09-03 02:05:46,186.141.200.121,2021-09-03 02:05:46
12188886,2021-09-03 02:24:41,Galaxi066,2021-09-03 02:24:41
12188909,2021-09-03 02:52:42,Xx.tkpro.xx,2021-09-03 02:52:42
12188924,2021-09-03 03:00:14,181.167.104.152,2021-09-03 03:00:14


In [16]:
logs.rename(columns={'username/ips':'user_blocked'},inplace=True)
#Solo me interesan aquellos usuarios que estan en user_blocked.
logs

,timestamp,action,title,user_blocked
0,2004-12-23 07:10:09,delete,Verificacionismo,Sabbut
1,2004-12-23 07:14:09,delete,Rapunzel,Sabbut
2,2004-12-23 07:26:10,delete,Macromedia Flash,Sabbut
3,2004-12-23 07:55:09,delete,Archivo:Gnu head.png,Sanbec
4,2004-12-23 08:02:22,delete,Archivo:Gnu-head-sm.jpg,Sanbec
...,...,...,...,...
12188941,2021-09-03 03:09:13,create,Usuario discusión:El guachicol,SeroBOT
12188942,2021-09-03 03:09:48,create,Usuario discusión:2806:2F0:3140:BE0:C09B:D1C4:...,Leonpolanco
12188943,2021-09-03 03:09:49,create,Usuario:Desiretjs,Desiretjs
12188944,2021-09-03 03:10:33,create,Contenedores,Correogsk


In [17]:
# Me quedo con los registros de las acciones de los usuarios que fueron bloqueados
blocked_logs = logs.loc[logs['user_blocked'].isin(blocked_actions.user_blocked)]
blocked_logs

,timestamp,action,title,user_blocked
0,2004-12-23 07:10:09,delete,Verificacionismo,Sabbut
1,2004-12-23 07:14:09,delete,Rapunzel,Sabbut
2,2004-12-23 07:26:10,delete,Macromedia Flash,Sabbut
3,2004-12-23 07:55:09,delete,Archivo:Gnu head.png,Sanbec
4,2004-12-23 08:02:22,delete,Archivo:Gnu-head-sm.jpg,Sanbec
...,...,...,...,...
12188864,2021-09-03 02:05:29,thank,Usuario:JonasBR,Albo Andrés R
12188870,2021-09-03 02:09:54,create,Hospital Metropolitano,Carlos yo
12188903,2021-09-03 02:41:29,thank,Usuario:Latothy99,JulioW4rrior00
12188905,2021-09-03 02:44:16,thank,Usuario:JulioW4rrior00,Latothy99


In [18]:
# Ahora que en blocked_logs tengo los usuarios que fueron bloqueados, le agrego la columna de la fecha que fueron por primera
# vez bloqueados
blocked_actions = blocked_actions[['user_blocked','first_block_date']]
before_being_blocked_logs = blocked_logs.merge(blocked_actions,on='user_blocked')
before_being_blocked_logs

,timestamp,action,title,user_blocked,first_block_date
0,2004-12-23 07:10:09,delete,Verificacionismo,Sabbut,2010-12-15 13:07:57
1,2004-12-23 07:14:09,delete,Rapunzel,Sabbut,2010-12-15 13:07:57
2,2004-12-23 07:26:10,delete,Macromedia Flash,Sabbut,2010-12-15 13:07:57
3,2004-12-24 09:51:09,delete,Ritmo,Sabbut,2010-12-15 13:07:57
4,2004-12-24 15:27:00,delete,Empresas actuales de Informática,Sabbut,2010-12-15 13:07:57
...,...,...,...,...,...
4879775,2021-09-02 18:44:32,create,Usuario:The Boys Studios,The Boys Studios,2021-09-03 00:18:14
4879776,2021-09-02 18:48:35,create,Noche De Odio,The Boys Studios,2021-09-03 00:18:14
4879777,2021-09-02 23:11:30,create,Usuario:Pervertidosinremedio,Pervertidosinremedio,2021-09-02 23:13:25
4879778,2021-09-02 23:14:35,create,Usuario:JohnWayne454545,JohnWayne454545,2021-09-03 00:39:46


In [19]:
# Ahora solo me quedo con aquellos que el timestamp es menor a su primera vez bloqueado.
before_being_blocked_logs['valid_log'] = before_being_blocked_logs['timestamp'] < before_being_blocked_logs['first_block_date']
before_being_blocked_logs

,timestamp,action,title,user_blocked,first_block_date,valid_log
0,2004-12-23 07:10:09,delete,Verificacionismo,Sabbut,2010-12-15 13:07:57,True
1,2004-12-23 07:14:09,delete,Rapunzel,Sabbut,2010-12-15 13:07:57,True
2,2004-12-23 07:26:10,delete,Macromedia Flash,Sabbut,2010-12-15 13:07:57,True
3,2004-12-24 09:51:09,delete,Ritmo,Sabbut,2010-12-15 13:07:57,True
4,2004-12-24 15:27:00,delete,Empresas actuales de Informática,Sabbut,2010-12-15 13:07:57,True
...,...,...,...,...,...,...
4879775,2021-09-02 18:44:32,create,Usuario:The Boys Studios,The Boys Studios,2021-09-03 00:18:14,True
4879776,2021-09-02 18:48:35,create,Noche De Odio,The Boys Studios,2021-09-03 00:18:14,True
4879777,2021-09-02 23:11:30,create,Usuario:Pervertidosinremedio,Pervertidosinremedio,2021-09-02 23:13:25,True
4879778,2021-09-02 23:14:35,create,Usuario:JohnWayne454545,JohnWayne454545,2021-09-03 00:39:46,True


In [20]:
before_being_blocked_logs = before_being_blocked_logs.loc[before_being_blocked_logs.valid_log == True]
before_being_blocked_logs = before_being_blocked_logs[['user_blocked','action','timestamp','first_block_date']]
before_being_blocked_logs

,user_blocked,action,timestamp,first_block_date
0,Sabbut,delete,2004-12-23 07:10:09,2010-12-15 13:07:57
1,Sabbut,delete,2004-12-23 07:14:09,2010-12-15 13:07:57
2,Sabbut,delete,2004-12-23 07:26:10,2010-12-15 13:07:57
3,Sabbut,delete,2004-12-24 09:51:09,2010-12-15 13:07:57
4,Sabbut,delete,2004-12-24 15:27:00,2010-12-15 13:07:57
...,...,...,...,...
4879775,The Boys Studios,create,2021-09-02 18:44:32,2021-09-03 00:18:14
4879776,The Boys Studios,create,2021-09-02 18:48:35,2021-09-03 00:18:14
4879777,Pervertidosinremedio,create,2021-09-02 23:11:30,2021-09-02 23:13:25
4879778,JohnWayne454545,create,2021-09-02 23:14:35,2021-09-03 00:39:46


In [21]:
# before_being_blocked_logs son las acciones de los usuarios que luego fueron bloqueados.